In [129]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
import json
def get_current_weather(location:str, unit:str="fahrenheit")->str:
    """Get the current weather in a given location"""
    if "karachi" in location.lower():
        return json.dumps({"location": "Karachi", "temperature": "10", "unit": "celsius"})
        return ({"location": "Karachi", "temperature": "10", "unit": "celsius"})

    elif "islamabad" in location.lower():
        return json.dumps({"location": "Islamabad", "temperature": "72", "unit": "fahrenheit"})
    elif "lahore" in location.lower():
        return json.dumps({"location": "Lahore", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


display(get_current_weather("karachi"))
display(get_current_weather("lahore"))


'{"location": "Karachi", "temperature": "10", "unit": "celsius"}'

'{"location": "Lahore", "temperature": "22", "unit": "celsius"}'

In [109]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_:bool = load_dotenv(find_dotenv())

client: OpenAI = OpenAI()

In [80]:
from openai.types.chat.chat_completion import ChatCompletion,ChatCompletionMessage

def weather_bot(prompt:str)-> ChatCompletion:
    response: ChatCompletion = client.chat.completions.create(
        messages= [
            {
                "role":"user",
                "content":prompt
            }

        ],
        model = "gpt-3.5-turbo-1106",
        tools = [
            {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. lahore, karachi",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                    },
                },
            }

        ],
        tool_choice ="auto" 
    )
    return response



# post-mortem when there is a need to call one function 
# As here we will only have Lahore "What is the weather like in Lahore "

In [48]:
response_single_function_call : ChatCompletionMessage =  weather_bot("What is the weather like in Lahore ")

In [49]:
display(dict((response_single_function_call).choices[0]))

{'finish_reason': 'tool_calls',
 'index': 0,
 'logprobs': None,
 'message': ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HnZJcH0Qpg5PLAhuzutTiXKr', function=Function(arguments='{"location":"lahore"}', name='get_current_weather'), type='function')])}

In [50]:
display(dict((response_single_function_call).choices[0].message))

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_HnZJcH0Qpg5PLAhuzutTiXKr', function=Function(arguments='{"location":"lahore"}', name='get_current_weather'), type='function')]}

In [51]:
display(dict((response_single_function_call).choices[0].message.tool_calls[0]))

{'id': 'call_HnZJcH0Qpg5PLAhuzutTiXKr',
 'function': Function(arguments='{"location":"lahore"}', name='get_current_weather'),
 'type': 'function'}

In [52]:
response_1 = (response_single_function_call).choices[0].message

response_1.tool_calls[0].function


Function(arguments='{"location":"lahore"}', name='get_current_weather')

In [53]:
response_1.tool_calls[0].function.arguments


'{"location":"lahore"}'

In [68]:
import json

#  json to python object conversion
response_python_object = json.loads(response_1.tool_calls[0].function.arguments)
print(response_python_object)


{'location': 'lahore'}


In [69]:
response_python_object["location"]

'lahore'

In [39]:
response_1.tool_calls

type(response_1.tool_calls)

list

In [40]:
list(response_1.tool_calls)
type(list(response_1.tool_calls))

list

In [26]:
display(dict((response_single_function_call).choices[0].message.tool_calls[0].function))

{'arguments': '{"location":"lahore"}', 'name': 'get_current_weather'}

# post-mortem when there is a need to call more than one function 
# As here we will have  Lahore and karachi both so we need to call two functions  "What is the weather like in Lahore and karachi  "
# Let's see the response and changes in tool_calls

In [41]:
response_multiple_function_call : ChatCompletionMessage =  weather_bot("What's the weather like in Lahore and Karachi?")


In [42]:
display(dict(response_multiple_function_call))

{'id': 'chatcmpl-9YqVpVIQtRg0rm87tyPGMzMxRBhSl',
 'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_aN6QjIxCbhQmZhjyMElH5KxN', function=Function(arguments='{"location": "lahore", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_WqHAQOYfJnDjnUBRu1bWbCj6', function=Function(arguments='{"location": "karachi", "unit": "celsius"}', name='get_current_weather'), type='function')]))],
 'created': 1718092037,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_482d920018',
 'usage': CompletionUsage(completion_tokens=57, prompt_tokens=85, total_tokens=142)}

In [44]:
display(dict(response_multiple_function_call.choices[0]))

{'finish_reason': 'tool_calls',
 'index': 0,
 'logprobs': None,
 'message': ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_aN6QjIxCbhQmZhjyMElH5KxN', function=Function(arguments='{"location": "lahore", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_WqHAQOYfJnDjnUBRu1bWbCj6', function=Function(arguments='{"location": "karachi", "unit": "celsius"}', name='get_current_weather'), type='function')])}

In [45]:
display(dict(response_multiple_function_call.choices[0].message))

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_aN6QjIxCbhQmZhjyMElH5KxN', function=Function(arguments='{"location": "lahore", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_WqHAQOYfJnDjnUBRu1bWbCj6', function=Function(arguments='{"location": "karachi", "unit": "celsius"}', name='get_current_weather'), type='function')]}

In [46]:
display(list(response_multiple_function_call.choices[0].message.tool_calls))

[ChatCompletionMessageToolCall(id='call_aN6QjIxCbhQmZhjyMElH5KxN', function=Function(arguments='{"location": "lahore", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_WqHAQOYfJnDjnUBRu1bWbCj6', function=Function(arguments='{"location": "karachi", "unit": "celsius"}', name='get_current_weather'), type='function')]

# Now the complete code 


In [130]:
from openai.types.chat.chat_completion import ChatCompletion,ChatCompletionMessage
import json 
def weather_bot(prompt:str)-> str:
    messages = [
            {
                "role":"user",
                "content":prompt
            }
        ]
    tools = [
            {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. lahore, karachi",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                    },
                },
            }
        ]
    response_1: ChatCompletion = client.chat.completions.create(
        messages= messages,
        model = "gpt-3.5-turbo-1106",
        tools = tools,
        tool_choice ="auto" 
    )
    response_1_msg = response_1.choices[0].message
    response_1_tool_calls = response_1_msg.tool_calls
    print(f"Response 1 tool calls: {list(response_1_tool_calls)}")

    if response_1_tool_calls:
        available_functions = {
            "get_current_weather" : get_current_weather
        }
        messages.append(response_1_msg)

        for tool_call in response_1_tool_calls:
            function_name = str(tool_call.function.name)
            function_to_call = available_functions.get(function_name)
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                function_args.get("location"),function_args.get("unit")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id , 
                    "role":"tool",
                    "name":function_name,
                    "content": function_response

                }
            )
    print(f"messages to get Second response  : {messages}")
    second_response: ChatCompletion = client.chat.completions.create(
        messages = messages,
        model = "gpt-3.5-turbo-1106"
    )
    display((f"Second response  : {dict(second_response)}"))
    return second_response.choices[0].message.content



        
    




In [131]:
weather_bot("What is the weather like in Lahore in fahrenheit ")

Response 1 tool calls: [ChatCompletionMessageToolCall(id='call_7d33vztqcx8iDdaBDcB3VeAd', function=Function(arguments='{"location":"lahore","unit":"fahrenheit"}', name='get_current_weather'), type='function')]
messages to get Second response  : [{'role': 'user', 'content': 'What is the weather like in Lahore in fahrenheit '}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7d33vztqcx8iDdaBDcB3VeAd', function=Function(arguments='{"location":"lahore","unit":"fahrenheit"}', name='get_current_weather'), type='function')]), {'tool_call_id': 'call_7d33vztqcx8iDdaBDcB3VeAd', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "Lahore", "temperature": "22", "unit": "celsius"}'}]


"Second response  : {'id': 'chatcmpl-9YtanQrJMNpApCxa9b5I8kyjoMOv5', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in Lahore is 22 degrees Celsius, which is approximately 71.6 degrees Fahrenheit.', role='assistant', function_call=None, tool_calls=None))], 'created': 1718103877, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_482d920018', 'usage': CompletionUsage(completion_tokens=21, prompt_tokens=69, total_tokens=90)}"

'The current temperature in Lahore is 22 degrees Celsius, which is approximately 71.6 degrees Fahrenheit.'

In [132]:
weather_bot("What is the weather like in Lahore and karachi in fahrenheit ")

Response 1 tool calls: [ChatCompletionMessageToolCall(id='call_8GwMfkMVCXPIWHi1aHq6G0yC', function=Function(arguments='{"location": "lahore", "unit": "fahrenheit"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_zU776p0rvHfSxRCH58nr8fyj', function=Function(arguments='{"location": "karachi", "unit": "fahrenheit"}', name='get_current_weather'), type='function')]
messages to get Second response  : [{'role': 'user', 'content': 'What is the weather like in Lahore and karachi in fahrenheit '}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8GwMfkMVCXPIWHi1aHq6G0yC', function=Function(arguments='{"location": "lahore", "unit": "fahrenheit"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_zU776p0rvHfSxRCH58nr8fyj', function=Function(arguments='{"location": "karachi", "unit": "fahrenheit"}', name='get_current_weather'), type='function')]), 

"Second response  : {'id': 'chatcmpl-9Ytaueggr4D5yT83hudO9hmqY9Unh', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Lahore is 22 degrees Celsius, which is approximately 71.6 degrees Fahrenheit. In Karachi, the current temperature is 10 degrees Celsius, which is approximately 50 degrees Fahrenheit.', role='assistant', function_call=None, tool_calls=None))], 'created': 1718103884, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_482d920018', 'usage': CompletionUsage(completion_tokens=41, prompt_tokens=131, total_tokens=172)}"

'The current weather in Lahore is 22 degrees Celsius, which is approximately 71.6 degrees Fahrenheit. In Karachi, the current temperature is 10 degrees Celsius, which is approximately 50 degrees Fahrenheit.'

In [135]:
weather_bot("What is the weather like in Lahore and islamabad. Give the resuts in Fahrenheit ")

Response 1 tool calls: [ChatCompletionMessageToolCall(id='call_iDjSEbYMrDZDmPDaEExJiy2U', function=Function(arguments='{"location": "lahore", "unit": "fahrenheit"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_WHcRBHRL6dhaKD2yURGO1O1o', function=Function(arguments='{"location": "islamabad", "unit": "fahrenheit"}', name='get_current_weather'), type='function')]
messages to get Second response  : [{'role': 'user', 'content': 'What is the weather like in Lahore and islamabad. Give the resuts in Fahrenheit '}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_iDjSEbYMrDZDmPDaEExJiy2U', function=Function(arguments='{"location": "lahore", "unit": "fahrenheit"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_WHcRBHRL6dhaKD2yURGO1O1o', function=Function(arguments='{"location": "islamabad", "unit": "fahrenheit"}', name='get_current_weather'

"Second response  : {'id': 'chatcmpl-9Ytgg4WL8OrFVWd2vieQBBeJwIkYH', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in Lahore is 22 degrees Celsius, which is approximately 71.6 degrees Fahrenheit. The current temperature in Islamabad is 72 degrees Fahrenheit.', role='assistant', function_call=None, tool_calls=None))], 'created': 1718104242, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_482d920018', 'usage': CompletionUsage(completion_tokens=32, prompt_tokens=137, total_tokens=169)}"

'The current temperature in Lahore is 22 degrees Celsius, which is approximately 71.6 degrees Fahrenheit. The current temperature in Islamabad is 72 degrees Fahrenheit.'